
# **Optimización de Memoria con Pandas**


Cuando trabajamos con grandes volúmenes de datos (cientos de miles o millones de filas), la biblioteca `pandas` puede consumir mucha memoria RAM si no se usan técnicas adecuadas.

Para ello deberemos:
- Verificar y optimizar el uso de memoria
- Usar tipos de datos más eficientes (`dtypes`)
- Cargar solo los datos necesarios
- Procesar archivos por bloques (chunks)
- Eliminar columnas innecesarias
- Convertir tipos de forma automática

## Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
import warnings
warnings.filterwarnings('ignore')

## Configurar el entorno de visualización

In [ ]:
def setup_matplotlib_for_plotting():
    """
    Setup matplotlib and seaborn for plotting with proper configuration.
    Call this function before creating any plots to ensure proper rendering.
    """
    # Configure matplotlib for non-interactive mode
    plt.switch_backend("Agg")

    # Set chart style
    plt.style.use("seaborn-v0_8")
    sns.set_palette("husl")

    # Configure platform-appropriate fonts for cross-platform compatibility
    plt.rcParams["font.sans-serif"] = ["Noto Sans CJK SC", "WenQuanYi Zen Hei", "PingFang SC", "Arial Unicode MS", "Hiragino Sans GB"]
    plt.rcParams["axes.unicode_minus"] = False

setup_matplotlib_for_plotting()

## Funciones auxiliares para métricas

In [ ]:
def get_memory_usage(df, label=""):
    """Obtiene el uso de memoria de un DataFrame"""
    memory_mb = df.memory_usage(deep=True).sum() / 1024**2
    if label:
        print(f"{label}: {memory_mb:.2f} MB")
    return memory_mb

def compare_memory_optimization(original_df, optimized_df, step_name):
    """Compara memoria entre DataFrames original y optimizado"""
    original_memory = get_memory_usage(original_df, f"Original ({step_name})")
    optimized_memory = get_memory_usage(optimized_df, f"Optimizado ({step_name})")
    
    savings_mb = original_memory - optimized_memory
    savings_percent = (savings_mb / original_memory) * 100
    
    print(f"Ahorro: {savings_mb:.2f} MB ({savings_percent:.1f}%)")
    print("-" * 50)
    
    return {
        'step': step_name,
        'original_mb': original_memory,
        'optimized_mb': optimized_memory,
        'savings_mb': savings_mb,
        'savings_percent': savings_percent
    }

def show_dtype_info(df, step_name=""):
    """Muestra información detallada de tipos de datos"""
    print(f"\nTipos de datos {step_name}:")
    dtype_summary = df.dtypes.value_counts()
    print(dtype_summary)
    
    # Mostrar tipos por columna
    print("\nDetalle por columna:")
    for col in df.columns:
        dtype = df[col].dtype
        unique_vals = df[col].nunique()
        memory_col = df[col].memory_usage(deep=True) / 1024**2
        print(f"  {col:20} | {str(dtype):10} | {unique_vals:>8} únicos | {memory_col:>6.2f} MB")

## Cargar un dataset grande

In [ ]:
# URLs de datasets
big_csv="https://github.com/ricardoahumada/Python_for_Data_Science/raw/refs/heads/master/data/2008.zip"
small_csv="https://github.com/ricardoahumada/Python_for_Data_Science/raw/refs/heads/master/data/2008_small.zip"
very_small_csv = 'https://github.com/ricardoahumada/data-for-auditors/raw/refs/heads/main/4.%20An%C3%A1lisis%20Masivo%20de%20Datos/Optimizacion/data/2008_very_small.csv'

In [ ]:
# Cargar dataset original
print("Cargando dataset original...")
start_time = time()
df_original = pd.read_csv(small_csv)
load_time = time() - start_time

print(f"Dataset cargado en {load_time:.2f} segundos")
print(f"Dimensiones: {df_original.shape[0]:,} filas × {df_original.shape[1]} columnas")

In [ ]:
# Información general del dataset
print("📋 Información del Dataset:")
df_original.info()

## Análisis inicial de memoria

In [ ]:
# Crear copia para trabajar
df = df_original.copy()

# Análisis inicial de memoria
print("ANÁLISIS INICIAL DE MEMORIA")
print("=" * 50)

baseline_memory = get_memory_usage(df, "Memoria inicial")

# Mostrar uso por columna
print("\nUso de memoria por columna:")
memory_by_column = df.memory_usage(deep=True).sort_values(ascending=False) / 1024**2
for col, memory in memory_by_column.head(10).items():
    print(f"  {col:25} | {memory:>8.2f} MB")

# Información de tipos inicial
show_dtype_info(df, "(Inicial)")

## OPTIMIZACIÓN 1: Cargar solo columnas necesarias

In [ ]:
# Mostrar todas las columnas disponibles
print("Columnas disponibles en el dataset:")
for i, col in enumerate(df_original.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nTotal de columnas: {len(df_original.columns)}")

In [ ]:
# Definir columnas más importantes (ejemplo para análisis de vuelos)
important_cols = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime',
       'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'ActualElapsedTime', 
       'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 
       'Distance', 'Cancelled', 'Diverted', 'CarrierDelay', 'WeatherDelay', 
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

# Verificar qué columnas existen realmente
available_cols = [col for col in important_cols if col in df_original.columns]
missing_cols = [col for col in important_cols if col not in df_original.columns]

print(f"Columnas a cargar: {len(available_cols)}")
print(f"Columnas faltantes: {len(missing_cols)}")
if missing_cols:
    print("  Faltantes:", missing_cols)

In [ ]:
# Comparación de memoria antes y después
df_step1 = df_original.copy()
result_step1 = compare_memory_optimization(
    df_step1, 
    pd.read_csv(small_csv, usecols=available_cols), 
    "Selección de Columnas"
)

# Actualizar DataFrame
df = pd.read_csv(small_csv, usecols=available_cols)
print(f"Nuevas dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")

## OPTIMIZACIÓN 2: Optimización manual de tipos de datos

In [ ]:
print("Optimización manual de tipos de datos")

# Analizar rangos de valores para optimización manual
print("\nAnálisis de rangos para optimización:")
for col in df.select_dtypes(include=['int64']).columns:
    min_val = df[col].min()
    max_val = df[col].max()
    print(f"  {col:20} | rango: [{min_val:>6}, {max_val:>6}]")

for col in df.select_dtypes(include=['float64']).columns:
    min_val = df[col].min()
    max_val = df[col].max()
    has_negative = df[col].min() < 0
    print(f"  {col:20} | rango: [{min_val:>8.1f}, {max_val:>8.1f}] | neg: {has_negative}")

print("\nAplicando optimizaciones manuales...")

In [ ]:
# Crear copia para comparar
df_step2_before = df.copy()

# Optimizaciones manuales basadas en el análisis
optimizations = {}

# Optimizar columnas numéricas basadas en rangos
for col in df.select_dtypes(include=['int64']).columns:
    col_min = df[col].min()
    col_max = df[col].max()
    
    if col_min >= 0 and col_max <= 255:
        df[col] = df[col].astype(np.uint8)
        optimizations[col] = "uint8"
    elif col_min >= 0 and col_max <= 65535:
        df[col] = df[col].astype(np.uint16)
        optimizations[col] = "uint16"
    elif col_min >= -128 and col_max <= 127:
        df[col] = df[col].astype(np.int8)
        optimizations[col] = "int8"
    elif col_min >= -32768 and col_max <= 32767:
        df[col] = df[col].astype(np.int16)
        optimizations[col] = "int16"
    elif col_min >= -8388608 and col_max <= 8388607:
        df[col] = df[col].astype(np.int32)
        optimizations[col] = "int32"

# Optimizar flotantes
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = pd.to_numeric(df[col], downcast='float')
    optimizations[col] = "float32"

# Convertir strings a categorías si tienen pocos valores únicos
for col in df.select_dtypes(include=['object']).columns:
    unique_ratio = df[col].nunique() / len(df)
    if unique_ratio < 0.5:  # Menos del 50% de valores únicos
        df[col] = df[col].astype('category')
        optimizations[col] = "category"

print("Optimizaciones aplicadas:")
for col, dtype in optimizations.items():
    print(f"  {col:20} → {dtype}")

In [ ]:
# Comparación después de optimización manual
result_step2 = compare_memory_optimization(
    df_step2_before, 
    df, 
    "Optimización Manual"
)

# Mostrar cambios en tipos
print("\nCambios en tipos de datos:")
for col in df.columns:
    if col in optimizations:
        print(f"  {col:20} | {str(df_step2_before[col].dtype):10} → {str(df[col].dtype):10}")

## OPTIMIZACIÓN 3: Función automática de optimización

In [ ]:
def optimize_dtypes_advanced(df):
    """
    Función avanzada de optimización de tipos de datos
    """
    optimized_df = df.copy()
    changes = {}
    
    # Optimizar números flotantes
    for col in df.select_dtypes(include=['float64']).columns:
        original_dtype = optimized_df[col].dtype
        optimized_df[col] = pd.to_numeric(df[col], downcast='float')
        new_dtype = optimized_df[col].dtype
        if original_dtype != new_dtype:
            changes[col] = f"{original_dtype} → {new_dtype}"
    
    # Optimizar enteros
    for col in df.select_dtypes(include=['int64']).columns:
        original_dtype = optimized_df[col].dtype
        optimized_df[col] = pd.to_numeric(df[col], downcast='integer')
        new_dtype = optimized_df[col].dtype
        if original_dtype != new_dtype:
            changes[col] = f"{original_dtype} → {new_dtype}"
    
    # Convertir objetos a categorías si es apropiado
    for col in df.select_dtypes(include=['object']).columns:
        # Solo convertir si hay pocos valores únicos relativamente
        unique_ratio = df[col].nunique() / len(df)
        if unique_ratio < 0.6:  # Umbral ajustable
            try:
                original_dtype = optimized_df[col].dtype
                optimized_df[col] = df[col].astype('category')
                new_dtype = optimized_df[col].dtype
                if original_dtype != new_dtype:
                    changes[col] = f"{original_dtype} → {new_dtype}"
            except:
                pass  # Si falla, mantener tipo original
    
    return optimized_df, changes

print("Aplicando función de optimización automática...")

In [ ]:
# Aplicar optimización automática
df_step3_before = df.copy()
df_optimized, auto_changes = optimize_dtypes_advanced(df)

print("Cambios automáticos aplicados:")
for col, change in auto_changes.items():
    print(f"  {col:20} | {change}")

# Actualizar DataFrame
df = df_optimized

In [ ]:
# Comparación después de optimización automática
result_step3 = compare_memory_optimization(
    df_step3_before, 
    df, 
    "Optimización Automática"
)

## OPTIMIZACIÓN 4: Limpieza de valores nulos

In [ ]:
# Analizar valores nulos
print("🔍 Análisis de valores nulos:")
null_counts = df.isnull().sum()
null_percentages = (null_counts / len(df)) * 100

null_info = pd.DataFrame({
    'Columna': null_counts.index,
    'Valores_Nulos': null_counts.values,
    'Porcentaje': null_percentages.values
}).sort_values('Valores_Nulos', ascending=False)

# Mostrar solo columnas con nulos
null_columns = null_info[null_info['Valores_Nulos'] > 0]
if len(null_columns) > 0:
    print(null_columns.to_string(index=False))
else:
    print("No hay valores nulos en el dataset")

In [ ]:
# Estrategia de limpieza según el tipo de datos
df_step4_before = df.copy()

print("\n🧹 Aplicando limpieza de valores nulos...")

# Para columnas numéricas, usar mediana (más robusta que media)
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df[col].isnull().any():
        fill_value = df[col].median()
        df[col].fillna(fill_value, inplace=True)
        print(f"  {col:20} | Rellenado con mediana: {fill_value}")

# Para columnas categóricas, usar moda (valor más frecuente)
categorical_cols = df.select_dtypes(include=['category', 'object']).columns
for col in categorical_cols:
    if df[col].isnull().any():
        fill_value = df[col].mode()[0] if len(df[col].mode()) > 0 else 'Desconocido'
        df[col].fillna(fill_value, inplace=True)
        print(f"  {col:20} | Rellenado con moda: {fill_value}")

# Verificar que no quedan nulos
remaining_nulls = df.isnull().sum().sum()
print(f"\nValores nulos restantes: {remaining_nulls}")

In [ ]:
# Comparación después de limpieza de nulos
result_step4 = compare_memory_optimization(
    df_step4_before, 
    df, 
    "Limpieza de Nulos"
)

## RESUMEN DE OPTIMIZACIONES

In [ ]:
# Crear tabla resumen de todas las optimizaciones
optimization_results = [result_step1, result_step2, result_step3, result_step4]

summary_df = pd.DataFrame(optimization_results)
print("RESUMEN COMPLETO DE OPTIMIZACIONES")
print("=" * 80)
print(summary_df.to_string(index=False, float_format='%.2f'))

# Calcular ahorros acumulativos
total_original = summary_df['original_mb'].iloc[0]
total_final = summary_df['optimized_mb'].iloc[-1]
total_savings_mb = total_original - total_final
total_savings_percent = (total_savings_mb / total_original) * 100

print(f"\nRESUMEN TOTAL:")
print(f"   Memoria inicial:     {total_original:>8.2f} MB")
print(f"   Memoria final:       {total_final:>8.2f} MB")
print(f"   Ahorro total:        {total_savings_mb:>8.2f} MB")
print(f"   Porcentaje de ahorro: {total_savings_percent:>7.1f}%")

## Información final del dataset optimizado

In [ ]:
print("INFORMACIÓN FINAL DEL DATASET OPTIMIZADO")
print("=" * 60)

df.info()

print(f"\nTipos de datos finales:")
show_dtype_info(df, "(Final)")

print(f"\nUso de memoria por columna (Top 10):")
final_memory = df.memory_usage(deep=True).sort_values(ascending=False) / 1024**2
for col, memory in final_memory.head(10).items():
    print(f"  {col:25} | {memory:>8.2f} MB")

## Ejemplo: Procesamiento por Chunks

In [ ]:
def process_chunk(chunk):
    """Ejemplo de función de procesamiento para chunks"""
    # Operaciones que podrías hacer en cada chunk
    stats = {
        'filas': len(chunk),
        'memoria_mb': chunk.memory_usage(deep=True).sum() / 1024**2,
        'columnas': len(chunk.columns)
    }
    return stats

print("📦 Procesamiento por Chunks - Demostración")
print("=" * 50)

chunk_results = []
chunk_size = 50000  # 50,000 filas por chunk

print(f"Procesando en chunks de {chunk_size:,} filas...")
for i, chunk in enumerate(pd.read_csv(small_csv, chunksize=chunk_size), 1):
    stats = process_chunk(chunk)
    stats['chunk_num'] = i
    chunk_results.append(stats)
    
    print(f"Chunk {i}: {stats['filas']:,} filas | {stats['memoria_mb']:.2f} MB")
    
    # Solo procesar los primeros 3 chunks para la demo
    if i >= 3:
        break

print(f"\nProcesados {len(chunk_results)} chunks")
print(f"Total memoria máxima por chunk: {max(r['memoria_mb'] for r in chunk_results):.2f} MB")

## Mejores Prácticas y Recomendaciones

1. **ANALIZA ANTES DE OPTIMIZAR:**
- Usa df.info() y df.memory_usage() para identificar problemas
- Revisa los rangos de valores para elegir tipos óptimos
- Identifica columnas con muchos valores nulos
   
2. **OPTIMIZA TIPOS DE DATOS:**
- int8/int16/int32 para enteros (según rango necesario)
- float32 para números decimales
- 'category' para strings con pocos valores únicos
- 'datetime64[ns]' para fechas
   
3. **LIMPIA TUS DATOS:**
- Elimina columnas que no uses con df.drop()
- Rellena valores nulos apropiadamente
- Usa usecols= para cargar solo columnas necesarias
   
4. **PARA ARCHIVOS MUY GRANDES:**
- Usa chunksize en pd.read_csv()
- Procesa datos por lotes
- Considera usar formatos más eficientes (Parquet, HDF5)
   
5. **MONITORIZA RENDIMIENTO:**
- Mide memoria antes y después de cada paso
- Usa time() para medir tiempo de procesamiento
- Documenta el impacto de cada optimización"


## Más información:

- [Documentación oficial de pandas](https://pandas.pydata.org/pandas-docs/stable/)
- [Categorías en pandas](https://pandas.pydata.org/docs/user_guide/enhancingperf.html)
- [Uso de chunks](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)
- [Downcasting numérico](https://pandas.pydata.org/docs/user_guide/enhancingperf.html#memory-usage)
- [Guía de optimización de memoria](https://pandas.pydata.org/docs/user_guide/scale.html)
- [Tipos de datos eficientes](https://pandas.pydata.org/docs/user_guide/enhancingperf.html#memory-usage)